## Import, upload, and create functions
In this file we walk through the steps to forming the tweets from out json files into dataframes

In [1]:
#In order to run Carmen, which is location application that we use to look at the location elements of the tweet that 
#are not readily apprent in the json. To use Carmen we have to first import this file.
import sys
!python setup.py install 

running install
running bdist_egg
running egg_info
writing carmen.egg-info/PKG-INFO
writing dependency_links to carmen.egg-info/dependency_links.txt
writing requirements to carmen.egg-info/requires.txt
writing top-level names to carmen.egg-info/top_level.txt
reading manifest file 'carmen.egg-info/SOURCES.txt'
writing manifest file 'carmen.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/carmen
copying build/lib/carmen/__init__.py -> build/bdist.linux-x86_64/egg/carmen
copying build/lib/carmen/names.py -> build/bdist.linux-x86_64/egg/carmen
creating build/bdist.linux-x86_64/egg/carmen/resolvers
copying build/lib/carmen/resolvers/place.py -> build/bdist.linux-x86_64/egg/carmen/resolvers
copying build/lib/carmen/resolvers/profile.py -> build/bdist.linux-x86_64/egg/carmen/resolvers
copying build/lib/carmen/resolvers/geocode.py -> build/bdist.linux-x8

In [2]:
#import numpy and then import carmen
import numpy as np
import carmen
#these are two commands that are needed to start carmen
resolver = carmen.get_resolver()
resolver.load_locations()

ModuleNotFoundError: No module named 'geopy'

Now we create a function to select the specific elements from the JSON that we need, as well as using Carmen in this function as well.

In [3]:
def flatten_tweets(tweets):
    """ Flattens out tweet dictionaries so relevant JSON is 
        in a top-level dictionary. """
    
    tweets_list = []
    # Iterate through each tweet
    for tweet_obj in tweets:
        tweet_obj = json.loads(tweet_obj)
        ''' User info'''
        # Store the user screen name in 'user-screen_name'
        tweet_obj['user-screen_name'] = tweet_obj['user']['screen_name']
        
        # Store the user location
        tweet_obj['user-location'] = tweet_obj['user']['location']
        
        # Store number of retweets
        tweet_obj['retweet_number'] = tweet_obj['retweet_count']
        
        # Store number of favorites
        tweet_obj['favorite_number'] = tweet_obj['favorite_count']
        
        # Store hashtags
        if len(tweet_obj['entities']['hashtags']) != 0:
            tweet_obj['hashtags'] = []
            for x in range(len(tweet_obj['entities']['hashtags'])):
                tweet_obj['hashtags'].append(tweet_obj['entities']['hashtags'][x]['text']) 
        if len(tweet_obj['entities']['hashtags']) == 0:
            tweet_obj['hashtags'] = 'None'
        
        # Store data
        tweet_obj['date'] = tweet_obj['created_at']
        
        #new location
        tweet_obj['location1'] = resolver.resolve_tweet(tweet_obj)
    
        ''' Text info'''
        # Check if this is a 140+ character tweet
        if 'extended_tweet' in tweet_obj:
            # Store the extended tweet text in 'extended_tweet-full_text'
            tweet_obj['extended_tweet-full_text'] = \
                                    tweet_obj['extended_tweet']['full_text']
        if 'quoted_status' in tweet_obj:
            # Store the retweet user screen name in 
            #'retweeted_status-user-screen_name'
            tweet_obj['quoted_status-user-screen_name'] = \
                            tweet_obj['quoted_status']['user']['screen_name']

            # Store the retweet text in 'retweeted_status-text'
            tweet_obj['quoted_status-text'] = \
                                            tweet_obj['quoted_status']['text']
    
            if 'extended_tweet' in tweet_obj['quoted_status']:
                # Store the extended retweet text in 
                #'retweeted_status-extended_tweet-full_text'
                tweet_obj['quoted_status-extended_tweet-full_text'] = \
                    tweet_obj['quoted_status']['extended_tweet']['full_text']
        
        ''' Place info'''
        if 'place' in tweet_obj:
            # Store the country code in 'place-country_code'
            try:
                tweet_obj['place-country'] = \
                                            tweet_obj['place']['country']
                
                tweet_obj['place-country_code'] = \
                                            tweet_obj['place']['country_code']
                
                tweet_obj['location-coordinates'] = \
                            tweet_obj['place']['bounding_box']['coordinates']
            except: pass
        
        tweets_list.append(tweet_obj)
        
    return tweets_list

In [5]:
#import json so that we can open the json file
import json
f = open('tweets.json')
tweets = f.readlines()

In [ ]:
import pandas as pd

# flatten tweets
tweets1 = flatten_tweets(tweets)

#here we create a list of the columns that we created above and designate these as the ones that we want in our datframe
columns = ['text', 'lang', 'user-location', 'place-country', 
           'place-country_code', 'location-coordinates', 
           'user-screen_name', 'retweet_number','hashtags','date', 'location1']
# Create a DataFrame from `tweets`
df_tweets = pd.DataFrame(tweets1, columns=columns)
df_tweets.head()

## Clean the dataframe
After we create this dataframe it is time to start parsing the locations so we can determine exactly where the tweets are from which will help us in the sorting process.

The first step is to sort through what Carmen created, we labeled this column as location1

In [8]:
#Carmen when it doesn't find a location for a value it labels it as None, we tried to sort out the values
#using the original keyword none, but for some reason this didn't work, so we instead turned to just using the 
#designated type for None which we've created a variable for here
none = type(None)

NoneType

In [11]:
#after creating that designated None value we can now create a function to find the city name if there is one
def city(place):
    #if there is some information about the place continue
    if type(place) != none:
        #if there is a city value return the city
        if place[1].city != '':
            return place[1].city
        #else return no
        else:
            return 'no'
    #else return no
    else:
        return 'no'
#apply the function and store the new value in the column city
df_tweets['city'] = df_tweets.apply(lambda row: city(row['location1']),axis=1)

<ipython-input-11-cbd001856326>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  use['city'] = use.apply(lambda row: city(row['location2']),axis=1)


In [12]:
#after creating that designated None value we can now create a function to find the state name if there is one
def state(place):
    #if there is some information about the place continue
    if type(place) != none:
        #if there is state return the state value
        if place[1].state != '':
            return place[1].state
        #else return no
        else:
            return 'no'
    #else return no
    else:
        return 'no'
#apply the function and store the new value in the column state
use['state'] = use.apply(lambda row: state(row['location1']),axis=1)

<ipython-input-12-6382041f244f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  use['state'] = use.apply(lambda row: state(row['location2']),axis=1)


In [14]:
#now we look at the tweets that Carmen didn't pick up because they were stored in the user created location
#and get their index
where = df_tweets[(use['user-location'] != '') & (use['city'] == 'no')]['user-location'].index
where

Int64Index([   12,   159,  4896,  5105,  5641,  5642,  5719,  5724,  5725,
             5726,
            ...
            18581, 18582, 18583, 18584, 18585, 18586, 18587, 18588, 18589,
            18590],
           dtype='int64', length=3305)

In [15]:
#now we sort through those indexes where there were user locations and store them in the city and state values
#that we made earlier
for x in where:
    #get the data from user location
    place = use.loc[x]['user-location']
    #if there multiple values in the user-location separate these values
    if ',' in place:
        #split the values
        s = place.split(',')
        #check the lenght of the list
        if len(s) == 2:
            #if it's two check if the second one is the US, if it's not then it's a state and city value
            if s[1] != ' US' or 'US':
                #assign the values 
                city = s[0]
                state = s[1]
                #check the values for ones that we know are already messed up
                if use.loc[x, 'city'] == 'Orange County' or "San Gabriel Valley":
                    use.loc[x,'city'] = 'Los Angeles'
                    use.loc[x,'state'] = 'California'
                if use.loc[x,'city'] == 'Sonoma County':
                    use.loc[x,'city'] = 'San Francisco'
                    use.loc[x,'state'] = 'California'
                #check the first value of city to see if there is a space, if there is one take the space out
                else:
                    if city[1] == ' ':
                        use.loc[x,'city'] = city[1:]
                    if state[1] == ' ':
                        use.loc[x,'state'] = state[1:]
                    else:
                        use.loc[x,'city'] = city
                        use.loc[x,'state'] = state
            elif s[1] == ' US' or 'US':
                state = s[0]
                use.loc[x,'state'] = state
    else:
        if place == 'DC via Nigeria':
            use.loc[x,'city'] = 'D.C.'
            use.loc[x,'state'] = 'D.C.'
        if place == 'City of Brotherly Love':
            use.loc[x,'city'] = 'Philadelphia'
            use.loc[x,'state'] = 'Pennsylvania'
        if place == 'SOUTH PHILADELPHIA—':
            use.loc[x,'city'] = 'Philadelphia'
            use.loc[x,'state'] = 'Pennsylvania'

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
